In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
import gpflow
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
gpflow.config.set_default_summary_fmt("notebook")
from gpflow.utilities import print_summary

In [3]:
data_path = '/Users/ScottJeen/OneDrive - University of Cambridge/research/Modelling/emerson_data/env'

# read in the pickled data
data = pd.read_pickle(data_path)

In [18]:
data

,COOLER COMP POWER (kW) 1,COOLER COMP POWER (kW) 2,COOLER COMP POWER (kW) 3,COOLER COMP POWER (kW) 4,COOLER COMP POWER (kW) 5,COOLER COMP POWER (kW) 6,COOLER HUMIDITY 1,COOLER HUMIDITY 2,COOLER HUMIDITY 3,COOLER HUMIDITY 4,...,GLYCOL PRESSURE 1,GLYCOL PRESSURE 2,GLYCOL TEMP 1,GLYCOL TEMP 2,OUTSIDE TEMP (oC),OUTSIDE DEW POINT (oC),OUTSIDE HUMIDITY (%),WIND DIR (DEGREES),WIND (km/h),PRESSURE (kPa)
Datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-15 11:00:00,4.683600,3.2436,4.5972,4.8204,0.0108,4.6512,0.6792,0.6682,0.9726,0.7004,...,1.844348,1.670255,10.360000,10.32,25.900,16.40,55.00,13.0,17.0,100.010
2020-07-15 11:03:00,4.640400,3.2328,4.6080,4.8384,0.0108,4.6764,0.6856,0.6690,0.9704,0.7010,...,1.842624,1.673702,10.340000,10.30,25.925,16.41,54.95,13.2,17.0,100.009
2020-07-15 11:06:00,4.773600,3.2292,4.6476,4.7592,0.0108,4.6836,0.6930,0.6652,0.9726,0.7010,...,1.842624,1.670255,10.340000,10.32,25.950,16.42,54.90,13.4,17.0,100.008
2020-07-15 11:09:00,4.640400,3.2256,4.6008,4.7196,0.0144,4.6512,0.6858,0.6644,0.9714,0.7008,...,1.839177,1.671979,10.320000,10.30,25.975,16.43,54.85,13.6,17.0,100.007
2020-07-15 11:12:00,4.723200,3.2292,4.6116,4.7592,0.0108,4.6836,0.6886,0.6642,0.9728,0.7030,...,1.840900,1.671979,10.320000,10.30,26.000,16.44,54.80,13.8,17.0,100.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01 12:27:00,0.014400,4.0788,3.8736,0.0036,0.0144,0.0072,0.6164,0.5998,0.6392,0.5978,...,1.737479,1.573728,10.170000,10.15,-6.900,-15.20,52.00,36.0,15.0,99.810
2021-02-01 12:30:00,0.010800,4.1040,3.8556,0.0108,0.0144,0.0072,0.6194,0.5966,0.6380,0.5966,...,1.735755,1.570281,10.170000,10.15,-6.900,-15.20,52.00,36.0,15.0,99.810
2021-02-01 12:33:00,3.924000,4.1184,3.8880,0.0036,0.0144,0.0072,0.6144,0.5930,0.6340,0.5946,...,1.737479,1.573728,10.170000,10.13,-6.900,-15.20,52.00,36.0,15.0,99.810


In [19]:
# cleaning function
def cleaning(data):
   # get cooler data
  cooler_cols = data.columns.str.contains('COOLER|DEW|OUTSIDE TEMP|OUTSIDE HUMIDITY|WIND|PRESSURE \(kPa\)', regex=True)
  cooler = data.iloc[:,cooler_cols]

  # get target matrix of temperatures
  target = cooler.iloc[:,[14]]
  features = cooler

  def normalise(df):
    from sklearn.preprocessing import MinMaxScaler
    # retrieve values
    vals = df.values

    # instantiate transformer
    transformer = MinMaxScaler(feature_range=(-1,1))

    # fit transformer
    transformer.fit(vals)

    # normalise
    norm = transformer.transform(vals)

    return norm, transformer

  X, X_trans = normalise(features)
  y, y_trans = normalise(target)

  # convert to np arrays
  X, y = np.asarray(X), np.asarray(y)

  return X, y, X_trans, y_trans

In [20]:
# clean the data
X, y, X_trans, y_trans = cleaning(data)

In [23]:
X.shape

(96483, 22)

## Build the GP Model

In [29]:
# build kernel
lengthscales = [0.3] * X.shape[1]

k = gpflow.kernels.Matern52(lengthscales=lengthscales)

# build model
model = gpflow.models.GPR(data=(X[:100], y[:100]), kernel=k, mean_function=None) # mean = 0
model.likelihood.variance.assign(0.01)

# # define optimizer
# opt = gpflow.optimizers.Scipy()
# opt_logs = opt.minimize(model.training_loss,\
#                         model.trainable_variables,
#                         options=dict(maxiter=100)
#                         )


<tf.Variable 'UnreadVariable' shape=() dtype=float64, numpy=-4.600266525158521>

In [30]:
print_summary(model)

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.0
GPR.kernel.lengthscales,Parameter,Softplus,,True,"(22,)",float64,"[0.3, 0.3, 0.3..."
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.01000000000000004
